

### 🏷️ **Credits & License**

* 🔗 [Chatterbox GitHub Repository](https://github.com/resemble-ai/chatterbox)
* 🤗 [Chatterbox on Hugging Face](https://huggingface.co/ResembleAI/chatterbox)
* 📄 **License**: Provided under the [MIT License](https://github.com/resemble-ai/chatterbox/blob/master/LICENSE)




### ⚠️ **Usage Disclaimer**

Use of this voice cloning model is subject to strict ethical and legal standards. By using this tool, you agree **not to** engage in any of the following prohibited activities:

* **Fraud or Deception**: Using cloned voices to create misleading or fraudulent content.
* **Impersonation**: Replicating someone’s voice without their explicit permission, especially for malicious, harmful, or deceptive purposes.
* **Illegal Activities**: Employing the model in any manner that violates local, national, or international laws and regulations.
* **Harmful Content Generation**: Creating offensive, defamatory, or unethical material, including content that spreads misinformation or causes harm.

> ⚖️ **Legal Responsibility**
> The developers of this tool disclaim all liability for misuse. **Users bear full responsibility** for ensuring that their usage complies with all applicable laws, regulations, and ethical guidelines.




In [ ]:
#@title Install ChatterBox
%cd /content/
# !git clone https://github.com/resemble-ai/chatterbox.git
!git clone https://github.com/NeuralFalconYT/chatterbox.git


def disable_watermark(file_path="./chatterbox/src/chatterbox/tts.py", disable=True):
    if not disable:
        print("Watermarking remains enabled. No changes made.")
        return

    with open(file_path, "r") as f:
        code = f.read()

    target_line = "watermarked_wav = self.watermarker.apply_watermark(wav, sample_rate=self.sr)"
    replacement_line = "watermarked_wav = wav"

    if target_line in code:
        code = code.replace(target_line, replacement_line)
        with open(file_path, "w") as f:
            f.write(code)
        print("✅ Watermarking disabled successfully.")
    else:
        print("ℹ️ Watermarking already disabled or line not found.")

def fix_chrome_hang(file_path = "./chatterbox/src/chatterbox/models/t3/t3.py"):
    try:
        with open(file_path, 'r') as f:
            content = f.read()

        content = content.replace(
            'for i in tqdm(range(max_new_tokens), desc="Sampling", dynamic_ncols=True):',
            'for i in range(max_new_tokens):'
        )

        with open(file_path, 'w') as f:
            f.write(content)

        print("✅ tqdm replaced with a normal for loop to fix Chrome hang.")
    except Exception as e:
        print(f"❌ Failed to patch file: {e}")
fix_chrome_hang()

## If you use your own voice recording and do NOT want the AI watermark,
## uncomment the following line to disable it:

# disable_watermark(disable=True)


%cd ./chatterbox
!wget https://raw.githubusercontent.com/NeuralFalconYT/chatterbox-colab/refs/heads/main/app.py

!mkdir /content/uploaded_audio
%cd /content/uploaded_audio
!wget https://storage.googleapis.com/chatterbox-demo-samples/prompts/female_shadowheart4.flac


!pip install numpy~=1.26.0
!pip install resampy==0.4.3
!pip install librosa==0.11.0
!pip install s3tokenizer
# !pip install torch==2.6.0
# !pip install torchaudio==2.6.0
!pip install transformers==4.46.3
!pip install diffusers==0.29.0
!pip install resemble-perth==1.0.1
!pip install omegaconf==2.3.0
!pip install conformer==0.3.2
!pip install safetensors==0.5.3
!pip install sentence-splitter
!pip install gradio>=5.9.1
!pip install soundfile
!pip install pysrt
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

Don't panic if you see this image.png red error cell, that just means Colab automatically restarted the kernel. Now, simply run the next cell.

In [1]:
#@title Gradio App
%cd /content/chatterbox
!python app.py --share #--debug

/content/chatterbox
2025-12-02 21:29:40.603360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764710980.622360    2203 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764710980.628285    2203 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764710980.643312    2203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764710980.643335    2203 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764710980.643339    2203 computation_placer.cc:177] co

#Run From Colab Cell

In [ ]:
#@title Mount Google Drive for auto save [ Optional, helpful for srt dubbing]
%cd /content/
from IPython.display import clear_output
from google.colab import drive
import os

auto_save_in_drive = False  # @param {type: "boolean"}

if auto_save_in_drive:
    drive.mount('/content/gdrive', force_remount=True)
    os.makedirs("/content/gdrive/MyDrive/colab_download", exist_ok=True)
    clear_output()
    print("✅ A copy of the cloned voice will be automatically saved to your Google Drive.")



/content


In [ ]:
#@title Download Model
%cd /content/chatterbox
# my useless funtions
import random
import numpy as np
import torch
import gradio as gr
import gc
import subprocess
# Initialize default pipeline
temp_audio_dir="./cloned_voices"
os.makedirs(temp_audio_dir, exist_ok=True)
def get_max_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory
        return round(total_memory / (1024 ** 3), 2)  # Convert to GB
    else:
        print("CUDA is not available.")
        return None

def is_gpu_memory_over_limit(safety_margin_gb=0.8):
    """
    Returns True if GPU memory usage exceeds (total - safety_margin).
    """
    max_memory_gb = get_max_gpu_memory()
    if max_memory_gb is None:
        return False  # Can't check memory if no GPU

    limit_gb = max_memory_gb - safety_margin_gb

    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        memory_used_mb_list = result.stdout.strip().splitlines()
        for i, memory_used_mb in enumerate(memory_used_mb_list):
            memory_used_gb = int(memory_used_mb) / 1024.0
            if memory_used_gb > limit_gb:
                print(f"Maximum GPU memory available: {max_memory_gb} GB")
                print(f"Memory limit set: {limit_gb} GB")
                print("⚠️ GPU memory usage has exceeded the safe threshold. Clearing memory and reloading the model...")
                return True
        # print("✅ GPU memory is within safe limits.")
        return False
    except Exception as e:
        print(f"Failed to check GPU memory: {e}")
        return False



from sentence_splitter import SentenceSplitter
import re
import uuid
import numpy as np
import wave
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os


def word_split(text, char_limit=300):
    words = text.split()
    chunks = []
    current_chunk = ""

    for word in words:
        if len(current_chunk) + len(word) + (1 if current_chunk else 0) <= char_limit:
            current_chunk += (" " if current_chunk else "") + word
        else:
            chunks.append(current_chunk)
            current_chunk = word

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def split_into_chunks(text, max_char_limit=300):
    if len(text)>=300:
      print("⚠️ The text is too long. Breaking it into smaller pieces so the voice generation works correctly.")

      splitter = SentenceSplitter(language='en')
      raw_sentences = splitter.split(text)

      # Flattened list of sentence-level word chunks
      sentence_chunks = []
      for sen in raw_sentences:
          sentence_chunks.extend(word_split(sen, char_limit=max_char_limit))

      chunks = []
      temp_str = ""

      for sentence in sentence_chunks:
          if len(temp_str) + len(sentence) + (1 if temp_str else 0) <= max_char_limit:
              temp_str += (" " if temp_str else "") + sentence
          else:
              chunks.append(temp_str)
              temp_str = sentence

      if temp_str:
          chunks.append(temp_str)

      return chunks
    else:
      return [text]


def clean_text(text):
    # Define replacement rules
    replacements = {
        "–": " ",  # Replace en-dash with space
        "—": " ",  #
        "-": " ",  # Replace hyphen with space
        "**": " ", # Replace double asterisks with space
        "*": " ",  # Replace single asterisk with space
        "#": " ",  # Replace hash with space
    }

    # Apply replacements
    for old, new in replacements.items():
        text = text.replace(old, new)

    # Remove emojis using regex (covering wide range of Unicode characters)
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|'  # Emoticons
        r'[\U0001F300-\U0001F5FF]|'  # Miscellaneous symbols and pictographs
        r'[\U0001F680-\U0001F6FF]|'  # Transport and map symbols
        r'[\U0001F700-\U0001F77F]|'  # Alchemical symbols
        r'[\U0001F780-\U0001F7FF]|'  # Geometric shapes extended
        r'[\U0001F800-\U0001F8FF]|'  # Supplemental arrows-C
        r'[\U0001F900-\U0001F9FF]|'  # Supplemental symbols and pictographs
        r'[\U0001FA00-\U0001FA6F]|'  # Chess symbols
        r'[\U0001FA70-\U0001FAFF]|'  # Symbols and pictographs extended-A
        r'[\U00002702-\U000027B0]|'  # Dingbats
        r'[\U0001F1E0-\U0001F1FF]'   # Flags (iOS)
        r'', flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    # Remove multiple spaces and extra line breaks
    text = re.sub(r'\s+', ' ', text).strip()

    return text




def tts_file_name(text, language="en"):
    global temp_audio_dir
    # Clean and process the text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = text.lower().strip().replace(" ", "_")

    # Ensure the text is not empty
    if not text:
        text = "audio"

    # Truncate to first 20 characters for filename
    truncated_text = text[:20]

    # Sanitize and format the language tag
    language = re.sub(r'\s+', '_', language.strip().lower()) if language else "unknown"

    # Generate random suffix
    random_string = uuid.uuid4().hex[:8].upper()

    # Construct the filename
    file_name = f"{temp_audio_dir}/{truncated_text}_{language}_{random_string}.wav"
    return file_name








def remove_silence_function(file_path,minimum_silence=50):
    # Extract file name and format from the provided path
    output_path = file_path.replace(".wav", "_no_silence.wav")
    audio_format = "wav"
    # Reading and splitting the audio file into chunks
    sound = AudioSegment.from_file(file_path, format=audio_format)
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=100,
                                    silence_thresh=-45,
                                    keep_silence=minimum_silence)
    # Putting the file back together
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk
    combined.export(output_path, format=audio_format)
    return output_path

#chatterbox code
from src.chatterbox.tts import ChatterboxTTS


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

chatterbox_model=None
def load_model():
    global chatterbox_model
    if chatterbox_model is None:
      del chatterbox_model
      gc.collect()
      torch.cuda.empty_cache()
    chatterbox_model = ChatterboxTTS.from_pretrained(DEVICE)
    return chatterbox_model

# for the first time
chatterbox_model=load_model()

import tempfile
import shutil
import os
import numpy as np
import soundfile as sf
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm


def generate(chatterbox_model, text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    if is_gpu_memory_over_limit():
        chatterbox_model = load_model()

    # Removed seed setting here to avoid conflicts
    # if seed_num != 0:
    #         set_seed(int(seed_num))

    wav = chatterbox_model.generate(
        text,
        audio_prompt_path=audio_prompt_path,
        exaggeration=exaggeration,
        temperature=temperature,
        cfg_weight=cfgw,
    )
    sr = chatterbox_model.sr
    audio = wav.squeeze(0).numpy()
    return sr, audio

def generate_and_save_all(text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    global chatterbox_model
    text = clean_text(text)
    chunks = split_into_chunks(text, max_char_limit=300)

    temp_dir = tempfile.mkdtemp(prefix="audio_chunks_")
    temp_files = []

    try:
        for idx, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Generating audio"):
            try:
                # Set seed once per chunk for reproducible variation
                if seed_num != 0:
                    set_seed(int(seed_num) + idx)

                sr, audio = generate(
                    chatterbox_model,
                    chunk,
                    audio_prompt_path,
                    exaggeration,
                    temperature,
                    seed_num=0,  # Pass 0 to skip seed setting inside generate
                    cfgw=cfgw
                )

                chunk_path = os.path.join(temp_dir, f"chunk_{idx:03}.wav")
                sf.write(chunk_path, audio, sr)
                temp_files.append(chunk_path)

            except Exception as e:
                print(f"⚠️ [Chunk {idx}] Generation failed: {e}")
                print(f"Text: {chunk}")
                print(f"Length: {len(chunk)}")
                continue  # Skip failed chunk

        # Merge all valid chunks
        final_audio = []
        for file_path in temp_files:
            try:
                data, _ = sf.read(file_path)
                final_audio.append(data)
            except Exception as e:
                print(f"💀 [Merging] Failed to read chunk: {file_path} ({e})")

        if final_audio:
            final_audio = np.concatenate(final_audio)
            final_path = tts_file_name(text, language="en")
            sf.write(final_path, final_audio, sr)
        else:
            raise RuntimeError("All audio chunk generations failed.")

    finally:
        shutil.rmtree(temp_dir)

    return final_path


from IPython.display import Audio, display

def play_audio(audio_path, autoplay=False):
    display(Audio(audio_path, autoplay=autoplay))



from IPython.display import clear_output
clear_output()
print("✅ The model has been downloaded and initialized successfully.")

✅ The model has been downloaded and initialized successfully.


In [ ]:
#@title Upload Reference Audio [Only use your own voice]
from google.colab import files
from IPython.display import clear_output

def upload_audio():
  upload_folder="/content/uploaded_audio"
  os.makedirs(upload_folder,exist_ok=True)
  os.chdir(upload_folder)
  f_names=[]
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(f"{upload_folder}/{fn}")
  os.chdir("/content/chatterbox")
  clear_output()
  return f_names[-1]
uploaded_audio=upload_audio()
play_audio(uploaded_audio, autoplay=False)
uploaded_audio


'/content/uploaded_audio/female_shadowheart4.flac'

In [ ]:
#@title Voice Clone
Text = "Now let's make my mum's favourite. So three mars bars into the pan. Then we add the tuna and just stir for a bit, just let the chocolate and fish infuse. A sprinkle of olive oil and some tomato ketchup. Now smell that. Oh boy this is going to be incredible."  # @param {type: "string"}

Reference_Audio = "/content/uploaded_audio/female_shadowheart4.flac"  # @param {type: "string"}

exaggeration = 0.5  # @param {type: "number"}
cfgw = 0.5  # @param {type: "number"}
temperature = 0.8  # @param {type: "number"}
seed_num = 0  # @param {type: "number"}
Remove_Silence= False  # @param {type: "boolean"}

cloned_voice_path=generate_and_save_all(Text, Reference_Audio, exaggeration, temperature, seed_num, cfgw)
if Remove_Silence:
  cloned_voice_path=remove_silence_function(cloned_voice_path,minimum_silence=50)
from IPython.display import clear_output
clear_output()
print(f"Cloned Voice Save at {os.path.abspath(cloned_voice_path)}")
drive_folder = "/content/gdrive/MyDrive/colab_download"

if os.path.exists(drive_folder):
    drive_path = shutil.copy(cloned_voice_path, drive_folder)
    print(f"✅ Cloned voice saved to Google Drive at: {drive_path}")

# cloned_voice_path=generate_and_save_all(chatterbox_model, text, "/content/Recording (5).wav", 0.5, 0.8, 0, 0.5)


#because colab can't handle large audio to play
if len(Text)<=2000:
  play_audio(cloned_voice_path, autoplay=True)
else:
  print("📥 The audio is too long to play automatically. Please download it to listen.")


Cloned Voice Save at /content/chatterbox/cloned_voices/now_lets_make_my_mum_en_A84A5CC5.wav


In [ ]:
#@title Download Audio

from google.colab import files
files.download(cloned_voice_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Compare Original and Cloned Voice [Don't play large audio on colab]

print("Original Voice:")
play_audio(Reference_Audio)
print("Cloned Voice:")
play_audio(cloned_voice_path)


Original Voice:


Cloned Voice:




### 🔧 **Chatterbox TTS Parameters**

| **Parameter**                       | **What It Does**                                                                                                       | **Easy Tips & Examples**                                                                                                 |
| ----------------------------------- | ---------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| **🎭 Exaggeration** (0.25–2.0)      | Controls how expressive or emotional the voice sounds. <br>Low = flat and robotic, High = dramatic and lively.         | 🟢 0.25 = boring, robotic voice<br>🟠 0.5 = normal, natural voice<br>🔴 1.5+ = excited, emotional, like acting in a play |
| **🧠 CFG / Pace** (0.0–1.0)         | Controls how closely the voice matches the reference audio style. <br>Low = more creative freedom, High = strict copy. | 🟢 0.0 = AI adds its own style<br>🔴 1.0 = copies reference voice exactly                                                |
| **🎲 Random Seed** (0 or number)    | Makes output repeatable. <br>Same seed + same text = same voice every time. <br>Seed=0 means random every time.        | 🟢 0 = surprise me, different result each time<br>🔢 123 = always get the same voice for the same text                   |
| **🔥 Temperature** (0.05–5.0)       | Controls how creative or “wild” the voice sounds. <br>Low = safe and predictable, High = more varied but can be weird. | 🟢 0.1 = calm, serious voice<br>🟠 0.8 = natural, clear voice<br>🔴 2.0+ = dramatic but might sound odd                  |
| **🧹 Remove\_Silence** (True/False) | Removes long pauses/silences from the generated voice audio. Helps speed up delivery.                                  | ✅ `True` = removes unnecessary silence for faster speech<br>❌ `False` = keeps all original pauses                        |

---

**Summary:**

* 🎭 **Exaggeration** = Emotion level
* 🧠 **CFG/Pace** = Stick to the style or improvise
* 🎲 **Seed** = Repeatable or random results
* 🔥 **Temperature** = Predictable vs. creative delivery
* 🧹 **Remove\_Silence** = Trim silence from generated audio



In [ ]:
#@title Upload  .SRT file
from google.colab import files
from IPython.display import clear_output

def upload_srt():
  upload_folder="/content/uploaded_srt"
  os.makedirs(upload_folder,exist_ok=True)
  os.chdir(upload_folder)
  f_names=[]
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(f"{upload_folder}/{fn}")
  os.chdir("/content/chatterbox")
  clear_output()
  return f_names[-1]
uploaded_srt=upload_srt()
uploaded_srt

'/content/uploaded_srt/example.srt'

In [ ]:
#@title 🎧 SRT Dubbing [Beta] , Auto Sync with Dynamic Speed Adjustment
import os
import re
import shutil
import subprocess
import uuid
import datetime
import platform
from pydub import AudioSegment
from tqdm.notebook import tqdm
import pysrt
import librosa
import soundfile as sf
from IPython.display import clear_output

# -------------------------------
# Utility Functions
# -------------------------------

def is_ffmpeg_installed():
    ffmpeg_exe = "ffmpeg.exe" if platform.system() == "Windows" else "ffmpeg"
    try:
        subprocess.run([ffmpeg_exe, "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        return True, ffmpeg_exe
    except Exception:
        return False, ffmpeg_exe

def speedup_audio_librosa(input_file, output_file, speedup_factor):
    try:
        y, sr = librosa.load(input_file, sr=None)
        y_stretched = librosa.effects.time_stretch(y, rate=speedup_factor)
        sf.write(output_file, y_stretched, sr)
    except Exception:
        shutil.copy(input_file, output_file)

def change_speed(input_file, output_file, speedup_factor, use_ffmpeg, ffmpeg_path):
    if use_ffmpeg:
        try:
            filters = []
            while speedup_factor > 2.0:
                filters.append("atempo=2.0")
                speedup_factor /= 2.0
            filters.append(f"atempo={speedup_factor}")
            atempo_filter = ",".join(filters)

            subprocess.run([
                ffmpeg_path, "-y", "-i", input_file,
                "-filter:a", atempo_filter,
                output_file
            ], check=True)
        except Exception:
            speedup_audio_librosa(input_file, output_file, speedup_factor)
    else:
        speedup_audio_librosa(input_file, output_file, speedup_factor)

def remove_edge_silence(input_path, output_path):
    y, sr = librosa.load(input_path, sr=None)
    trimmed_audio, _ = librosa.effects.trim(y, top_db=30)
    sf.write(output_path, trimmed_audio, sr)
    return output_path

def clean_srt(input_path):
    output_folder = "save_srt"
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, os.path.basename(input_path))

    def clean_line(text):
        for ch in ["[", "]", "♫", "\n"]:
            text = text.replace(ch, "")
        return text.strip()

    subs = pysrt.open(input_path)
    with open(output_path, "w", encoding="utf-8") as f:
        for sub in subs:
            f.write(f"{sub.index}\n")
            f.write(f"{sub.start} --> {sub.end}\n")
            f.write(f"{clean_line(sub.text)}\n\n")
    return output_path

def get_output_path(srt_file_path, language="en"):
    base_path = os.path.join(os.getcwd(), "TTS_DUB")
    os.makedirs(base_path, exist_ok=True)
    name = os.path.splitext(os.path.basename(srt_file_path))[0]
    unique_id = str(uuid.uuid4())[:6]
    return os.path.join(base_path, f"{name}_{language}_{unique_id}.wav").replace("__", "_")

def your_tts(output_path, actual_duration, text, reference_audio, exaggeration, temperature, seed, cfgw):
    cloned = generate_and_save_all(text, reference_audio, exaggeration, temperature, seed, cfgw)
    remove_edge_silence(cloned, output_path)

def safe_delete(filepath):
    try:
        os.remove(filepath)
        print(f"Deleted: {filepath}")
    except FileNotFoundError:
        print(f"File not found: {filepath}")
    except Exception as e:
        print(f"Error deleting {filepath}: {e}")

# -------------------------------
# SRT Dubbing Class
# -------------------------------

class SRTDubbing:
    def __init__(self):
        self.use_ffmpeg, self.ffmpeg_path = is_ffmpeg_installed()
        self.cache_dir = "./cache"
        os.makedirs("dummy", exist_ok=True)
        os.makedirs(self.cache_dir, exist_ok=True)

    @staticmethod
    def convert_to_millisecond(t):
        return t.hours * 3600000 + t.minutes * 60000 + t.seconds * 1000 + int(t.milliseconds)

    @staticmethod
    def read_srt_file(file_path):
        subs = pysrt.open(file_path, encoding='utf-8')
        entries = []
        prev_end = 0
        for idx, sub in enumerate(subs, 1):
            start = SRTDubbing.convert_to_millisecond(sub.start)
            end = SRTDubbing.convert_to_millisecond(sub.end)
            pause = start - prev_end if idx > 1 else start
            entries.append({
                'entry_number': idx,
                'start_time': start,
                'end_time': end,
                'text': sub.text.strip().replace("\n", " "),
                'pause_time': pause,
                'audio_name': f"{idx}.wav",
                'previous_pause': f"{idx}_before_pause.wav",
            })
            prev_end = end
        return entries


    def text_to_speech_srt(self, text, audio_path, actual_duration, reference_audio, exaggeration, temperature, seed, cfgw):
        # Step 1: Generate raw TTS to temp path
        temp = os.path.join(self.cache_dir, "temp_tts.wav")
        your_tts(temp, actual_duration, text, reference_audio, exaggeration, temperature, seed, cfgw)

        audio = AudioSegment.from_file(temp)

        # Step 2: If actual_duration is 0, just save directly
        if actual_duration == 0:
            shutil.move(temp, audio_path)
            safe_delete(temp)
            return

        # Step 3: If audio is longer than allowed, try removing silence first
        if len(audio) > actual_duration:
            temp = remove_silence_function(temp, minimum_silence=50)
            audio = AudioSegment.from_file(temp)

        # Step 4: Still longer? Apply speed adjustment
        if len(audio) > actual_duration:
            factor = len(audio) / actual_duration
            final_temp = os.path.join(self.cache_dir, "speedup_temp.wav")
            change_speed(temp, final_temp, factor, self.use_ffmpeg, self.ffmpeg_path)
            shutil.move(final_temp, audio_path)

        # Step 5: If shorter, pad with silence
        elif len(audio) < actual_duration:
            silence = AudioSegment.silent(duration=actual_duration - len(audio))
            padded = audio + silence
            padded.export(audio_path, format="wav")

        # Step 6: If perfect duration, save as is
        else:
            shutil.move(temp, audio_path)
        safe_delete(temp)
    def make_silence(self, duration, output_path):
        silence = AudioSegment.silent(duration=duration)
        silence.export(output_path, format="wav")
        return output_path

    def create_folder(self, srt_file_path):
        base = os.path.splitext(os.path.basename(srt_file_path))[0]
        tag = str(uuid.uuid4())[:4]
        folder = os.path.join("dummy", f"{base}_{tag}")
        os.makedirs(folder, exist_ok=True)
        return folder

    def concatenate_audio(self, audio_paths, output_path):
        final = AudioSegment.silent(duration=0)
        for p in audio_paths:
            final += AudioSegment.from_file(p)
        final.export(output_path, format="wav")

    def srt_to_dub(self, srt_file_path, output_path, reference_audio, exaggeration, temperature, seed, cfgw):
        entries = self.read_srt_file(srt_file_path)
        folder = self.create_folder(srt_file_path)
        join_list = []
        total_segments=len(entries)
        for count, i in enumerate(tqdm(entries, desc="Processing Subtitles"), 1):
            print(f"[{count}/{total_segments}] Processing: \"{i['text'][:50]} ...\"")
            text = i['text']
            duration = i['end_time'] - i['start_time']
            pause = i['pause_time']

            silence_path = os.path.join(folder, i['previous_pause'])
            self.make_silence(pause, silence_path)
            join_list.append(silence_path)

            audio_path = os.path.join(folder, i['audio_name'])
            self.text_to_speech_srt(text, audio_path, duration, reference_audio, exaggeration, temperature, seed, cfgw)
            join_list.append(audio_path)

            clear_output()

        self.concatenate_audio(join_list, output_path)

# -------------------------------
# Run Dubbing
# -------------------------------

def dubbing(srt_file_path, reference_audio, exaggeration=1.0, temperature=0.7, seed=42, cfgw=3.0):
    cleaner = clean_srt(srt_file_path)
    output = get_output_path(srt_file_path, "en")
    dubber = SRTDubbing()
    dubber.srt_to_dub(cleaner, output, reference_audio, exaggeration, temperature, seed, cfgw)
    return output

# -------------------------------
# Example Usage
# -------------------------------

srt_path = "/content/uploaded_srt/example.srt" # @param {type: "string"}
reference_audio = "/content/uploaded_audio/female_shadowheart4.flac"# @param {type: "string"}
exaggeration = 0.5 # @param {type: "number"}
cfgw = 0.5 # @param {type: "number"}
temperature = 0.8 # @param {type: "number"}
seed = 0 # @param {type: "number"}

result_path = dubbing(srt_path, reference_audio, exaggeration, temperature, seed, cfgw)
clear_output()
print(f"Dubbed audio saved at:\n{result_path}")

drive_folder = "/content/gdrive/MyDrive/colab_download"
if os.path.exists(drive_folder):
    drive_result_path = shutil.copy(result_path, drive_folder)
    print(f"Copied to Google Drive:\n{drive_result_path}")


Dubbed audio saved at:
/content/chatterbox/TTS_DUB/example_en_7699d6.wav


In [ ]:
#@title Download Dubbing Audio

from google.colab import files
files.download(result_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Play Dubbing Audio [Avoid large audio on colab]
play_audio(result_path)